In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()
chromosome = 1
query = (
    "SELECT *"
        "FROM `thegwensproject.genomiX.variants`"
    "WHERE chr_id = '" + str(chromosome) + "'"
    "LIMIT 20")
query_job = client.query(query)  # Make an API request.

print("The query data:")
for row in query_job:
    # Row values can be accessed by field name or index.
    print(row)

In [51]:
def count_variants(chr, pos):
    if pos == 0:
        query = (
            "SELECT chr_id_b37, position_b37, COUNT(*) as count FROM thegwensproject.genomiX.variants WHERE chr_id_b37 = '" +str(chr) +"' GROUP BY chr_id_b37, position_b37 HAVING COUNT(*) > 1 LIMIT 20;")
    else:
        query = (
            "SELECT chr_id_b37, position_b37, COUNT(*) as count FROM thegwensproject.genomiX.variants WHERE chr_id_b37 = '" +str(chr) +"' AND position_b37 = " + str(pos) + " GROUP BY chr_id_b37, position_b37 HAVING COUNT(*) > 1 LIMIT 20;")
    return query
query_job = client.query(count_variants(6, 166956723))
for row in query_job:
    # Row values can be accessed by field name or index.
    print(row)

In [50]:
def variant_info(chr, pos):
    query = (
        "SELECT * FROM thegwensproject.genomiX.variants WHERE chr_id_b37 = '" +str(chr) +"' AND position_b37 = " + str(pos) + " LIMIT 20;")
    return query

query_job = client.query(variant_info(6, 166956723))
for row in query_job:
    # Row values can be accessed by field name or index.
    print(row[9])
    print (row)

In [53]:
def tag_disease_variant_info(chr, pos):
    query = (
        "SELECT * FROM thegwensproject.genomiX.disease_variant_gene WHERE tag_chrom = '" +str(chr) +"' AND tag_pos = "+ str(pos) +"  LIMIT 20;")
    return query

query_job = client.query(tag_disease_variant_info(6, 166952906))
for row in query_job:
    # Row values can be accessed by field name or index.
    print(row['lead_pos'])
    print (row)

166956723
Row(('SAIGE_172', 'SAIGE', None, '2018-10-24', None, None, 'UKB SAIGE', True, 'Skin cancer', {'list': [{'element': 'MONDO_0002898'}]}, {'list': [{'element': 'European=408823'}]}, {'list': []}, 408823, 0, 13752, 'cell proliferation disorder', 41, '6', 166956723, 'A', 'C', '6', 166952906, 'T', 'C', 0.5981351856639999, 0.0, 0.0, 0.0, 1.0, 0.0, None, None, 1.0780997492739288, 1.0501526319677683, 1.1067906073868523, '+', None, None, None, 1.79, -8, 1.79e-08, 'ENSG00000071242', 'BLUEPRINT-T-CELL', 'eqtl', 'eqtl', {'list': []}, {'list': []}, None, None, 0.219065, 0.0415733, 4.57966e-07, 6.339166763394592, None, 0.3, None, None, None, None), {'study_id': 0, 'source': 1, 'pmid': 2, 'pub_date': 3, 'pub_journal': 4, 'pub_title': 5, 'pub_author': 6, 'has_sumstats': 7, 'trait_reported': 8, 'trait_efos': 9, 'ancestry_initial': 10, 'ancestry_replication': 11, 'n_initial': 12, 'n_replication': 13, 'n_cases': 14, 'trait_category': 15, 'num_assoc_loci': 16, 'lead_chrom': 17, 'lead_pos': 18, 'l

In [54]:
def lead_disease_variant_info(chr, pos):
    query = (
        "SELECT * FROM thegwensproject.genomiX.disease_variant_gene WHERE lead_chrom = '" +str(chr) +"' AND lead_pos = "+ str(pos) +"  LIMIT 20;")
    return query
query_job = client.query(lead_disease_variant_info(6, 166956723))
for row in query_job:
    # Row values can be accessed by field name or index.
    print(row['lead_pos'])
    print (row)

166956723
Row(('SAIGE_172', 'SAIGE', None, '2018-10-24', None, None, 'UKB SAIGE', True, 'Skin cancer', {'list': [{'element': 'MONDO_0002898'}]}, {'list': [{'element': 'European=408823'}]}, {'list': []}, 408823, 0, 13752, 'cell proliferation disorder', 41, '6', 166956723, 'A', 'C', '6', 166990944, 'C', 'T', 0.606862812196, 0.0, 0.0, 0.0, 1.0, 0.0, None, None, 1.0780997492739288, 1.0501526319677683, 1.1067906073868523, '+', None, None, None, 1.79, -8, 1.79e-08, 'ENSG00000213066', 'eQTLGen-UBERON_0000178', 'eqtl', 'eqtl', {'list': []}, {'list': []}, None, None, -0.09096568137197489, 0.00797287161217723, 3.7617e-30, 29.42461584290041, None, 0.7, None, None, None, None), {'study_id': 0, 'source': 1, 'pmid': 2, 'pub_date': 3, 'pub_journal': 4, 'pub_title': 5, 'pub_author': 6, 'has_sumstats': 7, 'trait_reported': 8, 'trait_efos': 9, 'ancestry_initial': 10, 'ancestry_replication': 11, 'n_initial': 12, 'n_replication': 13, 'n_cases': 14, 'trait_category': 15, 'num_assoc_loci': 16, 'lead_chrom':